## Simulations

In this notebook we wish to run a simulation demonstrating some of the basic claims we make regarding the random 
forest. 

The key claims we would like to demonstrate are thus:

- A dataset can have heirarchal behavior
    - an RF will identify such hierarchal structure 
    - an RF will capture local changes in covariance etc
    
    - A PCA CANNOT capture some of the effects that we will identify as local in distinct PCs.

- When a dataset undergoes changes in population prevalence, we identify this as a shift in factor values

- When a dataset undergoes a change in population behavior we identify this as a shift in predictive power

To reflect a hierarchal structure with meaningful local behavior, we will need several features that have different means among different clusters, but importantly also interact with each other, especially in different ways within different clusters. 

Let's operate on 10 features total. 

## On The Basis of Component Vectors

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import multivariate_normal,norm,beta
from sklearn.datasets import make_blobs
from sklearn.manifold import TSNE
from sklearn.preprocessing import scale


In [ ]:
# First we will generate the macro-structure. We will generate an eigenvector that applies globally, has a 
# multivariate normal set of loadings and a bimodal normal distribution of values

def generate(noise_multiplier=1):

    global_noise = [
        1,1,1
    ]

    factor_1 = [
        3,0,0
    ]
    
    factor_2 = [
        0,2,-2
    ]

    factor_3 = [
        0,2,2
    ]

    factors = np.vstack([factor_1,factor_2,factor_3])

    
    noise = multivariate_normal(global_noise,np.identity(3)*noise_multiplier).rvs(10000)    

    
    plt.figure()
    plt.imshow(factors,aspect='auto',interpolation='none')
    plt.show()
    
    loadings_1 = norm().rvs(10000) * 3
    loadings_2 = norm().rvs(2000)
    loadings_3 = norm().rvs(2000)
 

    loadings_1 = loadings_1[np.argsort(loadings_1)]
#     loadings_2 = loadings_2[np.argsort(loadings_2)]
#     loadings_3 = loadings_3[np.argsort(loadings_3)]

    combined_loadings = np.zeros((10000,3))
    combined_loadings[:,0] = loadings_1
    combined_loadings[-2000:,1] = loadings_2
    combined_loadings[:2000,2] = loadings_3
#     combined_loadings[sort_1[-2000:],1] = loadings_2
#     combined_loadings[sort_1[-2000:],2] = loadings_3
    
    
    coordinates = np.dot(combined_loadings,factors) + (noise * noise_multiplier)
    
    return (combined_loadings,factors,coordinates)

loadings,factors,coordinates = generate(noise_multiplier=1)
    

In [ ]:
# Let's take a look at the data we have generated. 
# We should have two broad clusters which are easily distinguished 

plt.figure()
plt.title("Example Loadings")
plt.imshow(loadings,aspect='auto',interpolation='none')
plt.colorbar()
plt.show()

plt.figure()
plt.title("Example Factor Values")
plt.imshow(factors,aspect='auto',interpolation='none')
plt.colorbar()
plt.show()

plt.figure()
plt.title("Example Feature Values")
plt.imshow(coordinates,aspect='auto',interpolation='none')
plt.colorbar()
plt.show()


In [ ]:
plt.figure()
plt.scatter(coordinates[:,1],coordinates[:,2])
plt.show()

In [ ]:
from scipy.cluster.hierarchy import linkage,dendrogram

# sample_agglomeration = dendrogram(linkage(coordinates, metric='cosine', method='average'), no_plot=True)['leaves']

plt.figure()
plt.title("Individual Feature Values")
plt.imshow(coordinates,aspect='auto',interpolation='none')
plt.colorbar()
plt.xlabel("Features")
plt.ylabel("Samples")
plt.show()

plt.figure()
plt.title("True Loadings")
plt.imshow(loadings,aspect='auto',interpolation='none')
plt.colorbar()
plt.xlabel("Factors")
plt.ylabel("Samples")
plt.show()

# plt.figure()
# plt.imshow(coordinates[sample_agglomeration],aspect='auto',interpolation='none')
# plt.show()

In [ ]:
# Now we will produce an embedding of the newly generated dataset for 
# easier visualization. 

t_coordinates = TSNE().fit_transform(coordinates)

plt.figure()
plt.title("TSNE Embedded Simulated Samples")
plt.scatter(*t_coordinates.T)
plt.show()



In [ ]:
# First we can visualize the true factor values in order to understand 
# which clusters are which

plt.figure()
plt.title("True Factor 1 Scores")
plt.scatter(*t_coordinates.T,c=loadings[:,0],cmap='bwr')
plt.colorbar()
plt.show()

plt.figure()
plt.title("True Factor 2 Scores")
plt.scatter(*t_coordinates.T,c=loadings[:,1],cmap='bwr')
plt.colorbar()
plt.show()

plt.figure()
plt.title("True Factor 3 Scores")
plt.scatter(*t_coordinates.T,c=loadings[:,2],cmap='bwr')
plt.colorbar()
plt.show()


In [ ]:
# Now we can perform a PCA analysis to see if we can recover the 
# global and local factors accurately. 

from sklearn.decomposition import PCA

model = PCA().fit(coordinates)

In [ ]:
# First we observe that PCA DOES explain most of the variance present in the datset,
# however it does so after using 4 components.

plt.figure()
plt.title("PC Explanatory Power (Ratio)")
plt.plot(model.explained_variance_ratio_)
plt.xlabel("PCs")
plt.ylabel("Variance Fraction Explained")
plt.show()

model.explained_variance_ratio_


In [ ]:
# However, we would be interested to see if it's possible to understand reover local 
# feature relationships in the PC loadings. After all, in an ideal case, we would recover
# our loadings exactly.

print(model.components_)


In [ ]:
# Here we see that the loadings of the PCs discovered do NOT contain a negative association 
# between 2 and the 6-7 pair. 

In [ ]:
pct = model.transform(coordinates)
pct.shape

In [ ]:
# We can observe directly that while the PCs recover the overall struture somewhat correctly,
# they make inappropriately global inferences about individual PCs.

# This occurs despite the fact that it is in principle perfectly possible to represent the 
# data structure corretly using 3 PCs with non-standardly distributed scores. 
# (As this was the generative process)

plt.figure()
plt.title("PC1 Scores")
plt.scatter(*t_coordinates.T,c=pct[:,0],cmap='bwr')
plt.colorbar()
plt.show()

plt.figure()
plt.title("PC2 Scores")
plt.scatter(*t_coordinates.T,c=pct[:,1],cmap='bwr')
plt.colorbar()
plt.show()

plt.figure()
plt.title("PC3 Scores")
plt.scatter(*t_coordinates.T,c=pct[:,2],cmap='bwr')
plt.colorbar()
plt.show()

plt.figure()
plt.title("PC4 Scores")
plt.scatter(*t_coordinates.T,c=pct[:,3],cmap='bwr')
plt.colorbar()
plt.show()

In [ ]:
plt.figure()
plt.title("Principal Component Loadings")
plt.imshow(pct[:,:3],interpolation='none',aspect='auto',cmap='bwr',vmin=-20,vmax=20)
plt.xlabel("PCs")
plt.ylabel("Samples")
plt.colorbar()
plt.show()

In [ ]:
# plt.figure()
# plt.scatter(true_factor_scores[:,1],pct[:,1])
# plt.show()

# plt.figure()
# plt.scatter(true_factor_scores[:,1],pct[:,2])
# plt.show()

# plt.figure()
# plt.scatter(true_factor_scores[:,1],pct[:,3])
# plt.show()

# plt.figure()
# plt.scatter(true_factor_scores[:,2],pct[:,1])
# plt.show()

# plt.figure()
# plt.scatter(true_factor_scores[:,2],pct[:,2])
# plt.show()

# plt.figure()
# plt.scatter(true_factor_scores[:,2],pct[:,3])
# plt.show()



In [ ]:
model = PCA(n_components=2).fit(coordinates)
pct = model.transform(coordinates)

recovered = model.inverse_transform(pct)

null_residuals = coordinates - np.mean(coordinates,axis=0)
recovered_residuals = coordinates - recovered

null_error = np.sum(np.power(null_residuals,2))
recovered_error = np.sum(np.power(recovered_residuals,2))

print(null_error)
print(recovered_error)

recovered_error / null_error

In [ ]:
# Now we can examine whether a Random Forest can capture the structure that eluded a PCA:

In [ ]:
import sys
sys.path.append('../src/')
import tree_reader as tr 
import lumberjack

In [ ]:
# We will train a relatively shallow forest, since the dataset is not complex. 
# We have relatively few features and would like a reliable structure, so will bootstrap a 
# large number of features per node (80% bootstrap)

# The rest of the parameters aren't deeply important, and so are left without comment. 

forest = lumberjack.fit(
    coordinates,
    trees=300,
    ifs=3,
    ofs=3,
    braids=1,
    ss=1000,
    leaves=10,
    depth=2,
    norm='l1',
    sfr=0,
#     reduce_input='true',
#     reduce_output='true',
    reduce_input='false',
    reduce_output='false',
)

In [ ]:
forest.tsne_coordinates = t_coordinates
forest.reset_split_clusters()
forest.interpret_splits(mode='additive_mean',metric='cosine',depth=2,resolution=.5,pca=False,k=100,relatives=True)
forest.maximum_spanning_tree(mode='samples')
# forest.most_likely_tree(depth=4)

In [ ]:
forest.html_tree_summary(n=5)

In [ ]:
# The resulting tree is available at:

# https://bx.bio.jhu.edu/track-hubs/bc/sc_summary/simulation/tree_template.html

In [ ]:
plt.figure()
plt.imshow(forest.factor_matrix()[:,1:],aspect='auto',interpolation='none')
plt.colorbar()
plt.show()



In [ ]:
plt.figure()
plt.scatter(true_factor_scores[:,1],forest.factor_matrix()[:,10])
plt.show()

plt.figure()
plt.scatter(true_factor_scores[:,1],forest.factor_matrix()[:,4])
plt.show()

plt.figure()
plt.scatter(true_factor_scores[:,2],forest.factor_matrix()[:,10])
plt.show()

plt.figure()
plt.scatter(true_factor_scores[:,2],forest.factor_matrix()[:,4])
plt.show()



In [ ]:
# Now we would like to range over a variety of noise levels

# noise_range = np.zeros((5,10000,10))
# true_range = np.zeros((5,10000,3))

# for i,noise in enumerate(range(-2,3)):
    
#     true_factor_scores = np.zeros((10000,3))
#     coordinates = np.zeros((10000,10))

#     true_factor_scores,coordinates = generate_global(
#         true_factor_scores,
#         coordinates,
#         noise_multiplier=2**noise,
#     )

#     true_factor_scores,coordinates = generate_local(
#         true_factor_scores,
#         coordinates,
#     )



#     plt.figure()
#     plt.title("Individual Feature Values")
#     plt.imshow(coordinates,aspect='auto',interpolation='none')
#     plt.colorbar()
#     plt.xlabel("Features")
#     plt.ylabel("Samples")
#     plt.show()
    
#     noise_range[i] = coordinates
#     true_range[i] = true_factor_scores


# plt.figure()
# plt.title("Individual Feature Values")
# plt.imshow(noise_range.reshape((50000,10)),aspect='auto',interpolation='none')
# plt.colorbar()
# plt.xlabel("Features")
# plt.ylabel("Samples")
# plt.show()

    
# t_coordinates = TSNE().fit_transform(noise_range.reshape((50000,10)))

# plt.figure()
# plt.title("TSNE Embedded Simulated Samples")
# plt.scatter(*t_coordinates.T)
# plt.show()

for i in range(5):
    plt.figure()
    plt.title("TSNE Embedded Simulated Samples")
    plt.scatter(*t_coordinates[i*10000:(i+1)*10000].T,c=np.arange(10000),cmap='rainbow')
    plt.show()
    
    
# for i in range(5):
    
#     forest = lumberjack.fit(
#         noise_range[i],
#         trees=300,
#         ifs=8,
#         ofs=8,
#         braids=1,
#         ss=1000,
#         leaves=10,
#         depth=4,
#         norm='l1',
#         sfr=0,
#     #     reduce_input='true',
#     #     reduce_output='true',
#         reduce_input='false',
#         reduce_output='false',
#     )
    
#     forest.tsne_coordinates = t_coordinates[i*10000:(i+1)*10000]
#     forest.reset_split_clusters()
#     forest.interpret_splits(mode='additive_mean',metric='cosine',depth=4,pca=3,k=500,relatives=True)
#     forest.maximum_spanning_tree(mode='samples')
    
#     plt.figure()
#     plt.imshow(forest.factor_matrix(),aspect='auto',interpolation='none',cmap='bwr')
#     plt.show()



In [ ]:
import sys
sys.path.append('../src/')
import tree_reader as tr 
import lumberjack

# We also want to quantify the best ability of both forest and PCA to capture Local Factor 2.
# We will quantify that by plotting the best-correlated factor of both the forest and the PCA
# over various ranges of noise value.

from scipy.spatial.distance import cdist

best_forest_correlations = []
best_pca_correlations = []

for i in range(5):

    forest = lumberjack.fit(
        noise_range[i],
        trees=300,
        ifs=8,
        ofs=8,
        braids=1,
        ss=1000,
        leaves=10,
        depth=4,
        norm='l1',
        sfr=0,
    #     reduce_input='true',
    #     reduce_output='true',
        reduce_input='false',
        reduce_output='false',
    )

    forest.interpret_splits(mode='additive_mean',metric='cosine',depth=4,pca=3,k=500,relatives=True)
    forest_factor_matrix = forest.factor_matrix()
    forest_correlations = cdist(true_range[i].T,forest_factor_matrix.T[1:],metric='correlation')
    best_forest_correlations.append(np.max(np.abs(forest_correlations-1),axis=1))
    
    print(f"{forest_correlations}")
    
    pca_model = PCA().fit(noise_range[i])
    pca_coordinates = pca_model.transform(noise_range[i])    
    pca_correlations = cdist(true_range[i].T,pca_coordinates.T,metric='correlation')
    best_pca_correlations.append(np.max(np.abs(pca_correlations-1),axis=1))
    
    print(f"{pca_correlations}")
    
print(best_forest_correlations)
print(best_pca_correlations)

In [ ]:
best_forest_correlations

In [ ]:
best_pca_correlations

In [ ]:
# Now that we have trained a forest, we should examine two ways in which an RFR can detet changes in the underlying 
# structure of a dataset: changes in behavior or changes in population composition.

# For this we will construct two new datasets, one with a shifted population, and one with shifted behavior.

# We will leave the global structure identical:

global_noise = [
    1,1,1,1,1,
    1,1,1,
    1,1,
]

loading_means_global = [
        1,0,-2,3,5,
        0,0,2,
        3,3
    ]
    
true_factor_scores = np.zeros((10000,3))
    
noise = multivariate_normal(global_noise,np.identity(10)/10).rvs(10000)    
loadings = multivariate_normal(loading_means_global,np.identity(10)/3).rvs(10000)

score_draws = norm().rvs(10000) / 3

score_draws[:3000] += 2
score_draws[3000:] += 5

true_factor_scores[:,0] = score_draws

coordinates = (loadings * np.tile(true_factor_scores[:,0],(10,1)).T) + noise


In [ ]:
# We will now copy that structure and create the population shifted local effects.

# The mean loadings are the same:

local_loading_means_1 = [
    0,2,0,0,0,
    1,3,1,
    0,2,
]

local_loading_means_2 = [
    0,-2,0,-2,0,
    1,3,3,
    3,0,
]

local_loadings_1 = multivariate_normal(local_loading_means_1,np.identity(10)/10).rvs(5000)
local_loadings_2 = multivariate_normal(local_loading_means_2,np.identity(10)/10).rvs(2000)

# However the factor scores we will draw will be different. 

true_factor_scores[3000:8000,1] = np.array(sorted((beta(.1,.1).rvs(5000) * 3 ) + 3))

# Note the asymmetric values for the beta distribution. 

true_factor_scores[8000:,2] = np.array(sorted((beta(.5,.2).rvs(2000) * 3 ) + 3))



local_coordinates_1 = np.tile(true_factor_scores[3000:8000,1],(10,1)).T * local_loadings_1
local_coordinates_2 = np.tile(true_factor_scores[8000:,2],(10,1)).T * local_loadings_2

coordinates[3000:8000] += local_coordinates_1
coordinates[8000:] += local_coordinates_2


In [ ]:
plt.figure()
plt.title("Individual Feature Values")
plt.imshow(coordinates,aspect='auto',interpolation='none')
plt.colorbar()
plt.xlabel("Features")
plt.ylabel("Samples")
plt.show()

plt.figure()
plt.title("True Factor Scores")
plt.imshow(true_factor_scores,aspect='auto',interpolation='none')
plt.colorbar()
plt.xlabel("Factors")
plt.ylabel("Samples")
plt.show()
